In [32]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
import tensorflow_probability as tfp
import h5py
import random
from sklearn.model_selection import train_test_split
import pandas as pd
from DataMaker import HDF5ImageGenerator

In [33]:
import photoz_utils

In [34]:
tfpl = tfp.layers
tf1 = tf.compat.v1
tfd = tfp.distributions

In [35]:
from tensorflow_probability.python.distributions import kl_divergence

In [36]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [37]:
LATENT_DIM = 16
IMAGE_SHAPE = (5, 127, 127)
BATCH_SIZE = 64
EPOCHS = 90
GB_LIMIT = 30
CHECKPOINTS_TO_SAVE = 1

In [38]:
args_gen = {'X_key': 'image',
    'y_key': 'specz_redshift',
    'scaler': True,
    'labels_encoding': False,
    'batch_size': BATCH_SIZE,
    'mode': 'train',
    'shuffle': True}

train_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training.hdf5', **args_gen)
val_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation.hdf5', **args_gen)
test_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5', **args_gen)

In [39]:
n = len(train_gen._indices)
t = len(test_gen._indices)
v = len(val_gen._indices)

In [40]:
def posterior_mean_field(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype = dtype, initializer = lambda shape, dtype: random_gaussian_initializer(shape, dtype), trainable = True),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc = t[ ..., : n],
                       scale = 0 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims = 1)),
    ])

def prior_trainable(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the prior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype = dtype),  # Returns a trainable variable of shape n, regardless of input
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc = t, scale = 0.1),
            reinterpreted_batch_ndims = 1)),
    ])

def random_gaussian_initializer(shape, dtype):
    n = int(shape / 2)
    loc_norm = tf.random_normal_initializer(mean = 0., stddev = 0.1)
    loc = tf.Variable(
        initial_value = loc_norm(shape = (n, ), dtype = dtype)
    )
    scale_norm = tf.random_normal_initializer(mean = -3., stddev = 0.1)
    scale = tf.Variable(
        initial_value = scale_norm(shape = (n, ), dtype = dtype)
    )
    return tf.concat([loc, scale], 0)

kl_divergence_function = lambda q, p, _: kl_divergence(q, p) / n

In [41]:
import keras
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

In [42]:
model_variational = Sequential([
    InputLayer(input_shape = (5, 127, 127)),
    tfpl.Convolution2DFlipout(32, kernel_size=(2, 2), activation = 'tanh', kernel_divergence_fn = kl_divergence_function, data_format = 'channels_first'),
    MaxPooling2D(2, 2, data_format = 'channels_first'),
    Conv2D(32, kernel_size = (2, 2), activation = 'tanh', data_format = 'channels_first'),
    MaxPooling2D(2, 2, data_format = 'channels_first'),
    Conv2D(32, kernel_size = (2, 2), activation = 'relu', data_format = 'channels_first'),
    Conv2D(32, kernel_size = (2, 2), activation = 'relu', data_format = 'channels_first'),
    Flatten(),
    Dropout(0.2),
    tfpl.DenseVariational(512, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = 1 / n),
    tfpl.DenseVariational(128, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = 1 / n),
    Dense(tfpl.IndependentNormal.params_size(1)),
    tfpl.IndependentNormal(1)
])

model_variational.build((n, 5, 127, 127))

model_variational.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_flipout_1 (Conv2DFli  (None, 32, 126, 126)     1312      
 pout)                                                           
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 63, 63)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 62, 62)        4128      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 31, 31)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 30, 30)        4128      
                                                      

In [43]:
def loss_function(targets, estimated_distribution):
    return - estimated_distribution.log_prob(targets) / (1 + targets)

model_variational.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = loss_function, metrics = [keras.metrics.RootMeanSquaredError()])

In [44]:
import tensorboard

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

model_name = 'HSC_v6_BCNN_full'

checkpoint_filepath = os.path.join('/models/', model_name)

weights_path = os.path.join('/models/', model_name) + '_weights'

log_directory = os.path.join('/logs/', model_name)

logs_callback = TensorBoard(log_dir = log_directory)

weights_callback = ModelCheckpoint(filepath = os.path.join(checkpoint_filepath, 'weights_epoch{epoch}.hdf5'),
                                   save_freq = int(EPOCHS / CHECKPOINTS_TO_SAVE) * int(n / BATCH_SIZE),
                                   save_weights_only = True, save_best_only=True)

LR_callback = ReduceLROnPlateau()

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_directory, histogram_freq = 1)

In [46]:
model_variational.fit(train_gen, epochs = 1, shuffle = True, verbose = 1, validation_data = val_gen, callbacks = [tensorboard_callback, logs_callback, LR_callback, weights_callback])

Layer DenseVariational has arguments ['self', 'units', 'make_posterior_fn', 'make_prior_fn', 'kl_weight', 'kl_use_exact', 'activation', 'use_bias', 'activity_regularizer']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Layer DenseVariational has arguments ['self', 'units', 'make_posterior_fn', 'make_prior_fn', 'kl_weight', 'kl_use_exact', 'activation', 'use_bias', 'activity_regularizer']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
   

2022-09-13 00:01:03.726403: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-09-13 00:01:03.726456: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.
2022-09-13 00:01:03.817188: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-09-13 00:01:03.817219: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Graph execution error:

Detected at node 'sequential_1/conv2d_flipout_1/Conv2D' defined at (most recent call last):
    File "/home/billyli/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/billyli/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/billyli/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/billyli/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/billyli/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/billyli/miniconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/billyli/miniconda3/lib/pyt

In [15]:
model_variational.save_weights(weights_path.format(epoch = 90))

In [23]:
model_variational.load_weights(weights_path)

In [17]:
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training.hdf5', 'r')
y_train = np.asarray(hf_train['specz_redshift'][0:])[..., None]
object_id_train = np.asarray(hf_train['object_id'][0:])
hf_train.close()

In [27]:
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5', 'r')
x_test = np.asarray(np.transpose(hf_test['image'][0:],(0,1,2,3)))
y_test = np.asarray(hf_test['specz_redshift'][0:])[..., None]
object_id = np.asarray(hf_test['object_id'][0:])
hf_test.close()

In [19]:
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation.hdf5', 'r')
y_validation = np.asarray(hf_validation['specz_redshift'][0:])[..., None]
object_id_validation = np.asarray(hf_validation['object_id'][0:])
hf_validation.close()

In [28]:
n = 10000
t = 2000
v = 2000

In [30]:
model_variational(np.expand_dims(x_test[0], axis = 0)).mean()[0][0]

2022-09-12 23:23:30.195754: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-09-12 23:23:30.195821: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer "conv2d_flipout" (type Conv2DFlipout).

DNN library is not found. [Op:Conv2D]

Call arguments received by layer "conv2d_flipout" (type Conv2DFlipout):
  • inputs=tf.Tensor(shape=(1, 5, 127, 127), dtype=float32)

In [29]:
photoz = []

for i in range(t):
    photoz.append(model_variational(np.expand_dims(x_test[i], axis = 0)).mean()[0][0])

photoz = np.asarray(photoz).astype('float32')

ValueError: Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 5, 127, 127), found shape=(5, 127, 127)

In [26]:
photoz = []

for i in range(t):
    photoz.append(model_variational(np.array([test_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz = np.asarray(photoz).astype('float32')

2022-09-12 23:19:12.804195: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-09-12 23:19:12.804266: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer "conv2d_flipout" (type Conv2DFlipout).

DNN library is not found. [Op:Conv2D]

Call arguments received by layer "conv2d_flipout" (type Conv2DFlipout):
  • inputs=tf.Tensor(shape=(1, 5, 127, 127), dtype=float32)

photoz_train = []

for i in range(n):
    photoz_train.append(model_variational(np.array([train_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz_train = np.asarray(photoz_train).astype('float32')

photoz_validation = []

for i in range(v):
    photoz_validation.append(model_variational(np.array([val_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz_validation = np.asarray(photoz_validation).astype('float32')

In [ ]:
specz = np.asarray(y_test)[:,0][:t]
# specz_train = np.asarray(y_train)[:,0][:n]
# specz_validation = np.asarray(y_validation)[:,0][:v]

In [ ]:
photoz_err = []

for i in range(t):
    photoz_err.append(model_variational(np.array([test_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

photoz_err_train = []

for i in range(n):
    photoz_err_train.append(model_variational(np.array([train_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

photoz_err_validation = []

for i in range(v):
    photoz_err_validation.append(model_variational(np.array([val_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

In [ ]:
from scipy.stats import gaussian_kde
xy = np.asarray(np.vstack([specz, photoz])).astype('float32')
z = gaussian_kde(xy)(xy)

In [ ]:
import seaborn as sns
z_max = 4
sns.set(rc={'figure.figsize': (10, 10), 'lines.markersize': 20})
plt.rcParams.update({'font.size': 40})
fig, ax = plt.subplots()
scatter_plot = ax.scatter(specz, photoz, c = z, s = 1, edgecolor = None)
plt.colorbar(scatter_plot, label = 'Density')
plt.title('Prediction')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max],color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
plt.show()

sns.set(rc={'figure.figsize':(10, 10), 'lines.markersize':20})
plt.rcParams.update({'font.size': 100})
sns.set(font_scale=3)

data = {'BCNN':10.7, 'BNN':19.7, 'Mizuki':27.4,
        'DEmP':25}
models_list = list(data.keys())
outlier_rates = list(data.values())

plt.bar(models_list, outlier_rates, color=('blue', 'red', 'red', 'red'))
plt.plot([-.5, 3.5], [15, 15], color = 'black', linestyle = 'dashed')
plt.title("Outlier Rates")
plt.ylabel("% Outlier Rate")
plt.xlabel("Model Type")
plt.show()

In [ ]:
overlap_variational = 0
overlap_array_variational = []
for i in range(0,t):
    if np.abs(specz[i]-photoz[i])<=photoz_err[i]:
        overlap_variational += 1
        overlap_array_variational.append(1)
    else:
        overlap_array_variational.append(0)

In [ ]:
fig, ax = plt.subplots()
display_size = 1000
scatter_plot = ax.scatter(specz[:display_size], photoz[:display_size], c = z[:display_size], s = 4, edgecolor = None, zorder = 2)
error_plot = ax.errorbar(specz[:display_size], photoz[:display_size], yerr = photoz_err[:display_size], fmt = "o", markersize = 0, color = 'blue', elinewidth = 1, zorder = 1)
plt.title('Prediction with error, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max], color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
print("Coverage: "+str(overlap_variational/t))

In [ ]:
combined_array_variational = np.transpose(np.vstack((specz, photoz, photoz_err, overlap_array_variational))) 

In [ ]:
sorted_combined_array_variational = sorted(combined_array_variational, key=lambda x: x[0])

In [ ]:
bins_variational = int(z_max*5)
splitted_sorted_combined_array_variational = np.array_split(sorted_combined_array_variational, bins_variational)
coverage_variational = []
for i in range(0, bins_variational):
    bins_count_variational = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        if splitted_sorted_combined_array_variational[i][j][3] == 1:
            bins_count_variational += 1
    coverage_variational.append(bins_count_variational/len(splitted_sorted_combined_array_variational[i])/0.6827)
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
plt.plot(x_array, coverage_variational, c = 'red')
plt.title('Coverage, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('coverage')
plt.ylim([0, 1.5])
plt.plot([0, 4], [1, 1], color = 'black', linestyle = 'dashed')
print("Coverage: "+str(overlap_variational/t))

In [ ]:
mean_array = []
for i in range(0, bins_variational):
    total = 0
    for j in range(0, len(splitted_sorted_combined_array_variational[i])):
        bias = (splitted_sorted_combined_array_variational[i][j][1]-splitted_sorted_combined_array_variational[i][j][0])
        total += bias/(1 + splitted_sorted_combined_array_variational[i][j][0])
    mean_array.append(total/len(splitted_sorted_combined_array_variational[i]))
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
error_array = []
for i in range(0,bins_variational):
    error_total = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        error_total += splitted_sorted_combined_array_variational[i][j][2]
    error_array.append(error_total/len(splitted_sorted_combined_array_variational[i]))

In [ ]:
sns.set(rc={'figure.figsize':(5, 5), 'lines.markersize':40})
plt.rcParams.update({'font.size': 40})
plt.errorbar(x_array, mean_array, yerr = error_array, fmt = "o", color = 'blue', markersize = 4, elinewidth = 2)
plt.title('Bias Plot for Probabilistic Model')
plt.xlabel('True Redshift')
plt.ylabel('Bias (Predicted - True)')
plt.plot([0, z_max], [0, 0], color = 'black')
plt.plot([0, z_max], [0.02, 0.02], color = 'black', linestyle = 'dashed')
plt.plot([0, z_max], [-0.02, -0.02], color = 'black', linestyle = 'dashed')
plt.ylim([-0.5, 0.5])
plt.show()

In [ ]:
combined_outlier_array = np.transpose(np.vstack((specz,np.subtract(photoz, specz))))
sorted_combined_outlier_array = sorted(combined_outlier_array, key=lambda x: x[0])
bins_outlier = int(z_max*10)
splitted_sorted_combined_outlier_array = np.array_split(sorted_combined_outlier_array,bins_outlier)
outlier_array = []
outlier_count_total = 0
for i in range(0,bins_outlier):
    outlier_count = 0
    for j in range(0,len(splitted_sorted_combined_outlier_array[i])):
        if np.abs(splitted_sorted_combined_outlier_array[i][j][1]/(splitted_sorted_combined_outlier_array[i][j][0] + 1)) >= .15:
            outlier_count += 1
            outlier_count_total += 1
    outlier_array.append(outlier_count/len(specz)*bins_outlier)
x_array_outlier = np.arange(0,z_max,z_max/bins_outlier)
plt.plot(x_array_outlier, outlier_array, c = 'blue')
plt.title('Outlier Rate')
plt.xlabel('spec z')
plt.ylabel('outlier rate')
plt.xlim([0,z_max])
plt.ylim([0,1/2])
plt.show()

print("Outlier Rate: "+str(outlier_count_total/len(specz)))

In [ ]:
photoz_utils.save_with_oid_std(model_name, object_id, specz, photoz, photoz_err)
# photoz_utils.save_train_with_oid_std(model_name, object_id_train, specz_train, photoz_train, photoz_err_train)
# photoz_utils.save_validation_with_oid_std(model_name, object_id_validation, specz_validation, photoz_validation, photoz_err_validation)

In [ ]:
uncertains = []
for i in range(0, len(photoz_err)):
    if photoz_err[i].numpy() >= 1:
        uncertains.append(i)
        print("Object ID: " + str(object_id[i]) + ", Spectro_z: " + str(specz[i]) + ", Photo_z: " + str(photoz[i]) + ", Photo_z_err: " + str(photoz_err[i].numpy()))

In [ ]:
check = 2
current = uncertains[check]
print("Object ID: " + str(object_id[current]) + ", Spectro_z: " + str(specz[current]) + ", Photo_z: " + str(photoz[current]) + ", Photo_z_err: " + str(photoz_err[current].numpy()))
plt.imshow(x_test[current][:,:,0])
plt.grid(False)

In [ ]:
biases = []
for i in range(0, len(photoz)):
    if np.abs(photoz[i]-specz[i]) >= 1 and photoz_err[i].numpy() <= 0.5:
        biases.append(i)
        print("Object ID: " + str(object_id[i]) + ", Spectro_z: " + str(specz[i]) + ", Photo_z: " + str(photoz[i]) + ", Photo_z_err: " + str(photoz_err[i].numpy()))

In [ ]:
check_bias = 3
current_bias = biases[check_bias]
print("Object ID: " + str(object_id[current_bias]) + ", Spectro_z: " + str(specz[current_bias]) + ", Photo_z: " + str(photoz[current_bias]) + ", Photo_z_err: " + str(photoz_err[current_bias].numpy()))
plt.imshow(x_test[current_bias][:,:,0])
plt.grid(False)

In [ ]:
# END